In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import json
import fiona
import glob
from shapely.geometry import Point

from methods import read_shst_extract

In [2]:
data_interim_dir = "../../data/interim/"

In [3]:
link_file = data_interim_dir + "step3_join_shst_extraction_with_osm/" + "link.json"
with open(link_file) as f:
    link_json = json.load(f)
link_df = pd.DataFrame(link_json)

shape_gdf = gpd.read_file(data_interim_dir + "step3_join_shst_extraction_with_osm/" 
                          + "shape.geojson")

link_df = pd.merge(link_df,
                   shape_gdf[["id", "geometry"]],
                   how = "left",
                   left_on = "shstGeometryId",
                   right_on = "id")

link_df = gpd.GeoDataFrame(link_df, geometry = link_df["geometry"],
                            crs={"init" : "epsg:4326"})

In [4]:
conflation_df = pd.read_csv("../../data/interim/conflation_result.csv")

C:\Users\wangs1\.conda\envs\MetC_new\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
conflation_df.columns

Index(['shstReferenceId', 'roadway', 'lanes_osm', 'drive_access',
       'bike_access', 'walk_access', 'tomtom_FRC', 'tomtom_FRC_def',
       'tomtom_lanes', 'tomtom_unique_id', 'F_JNCTID', 'T_JNCTID',
       'tomtom_name', 'tomtom_shieldnum', 'tomtom_rtedir', 'TM2Marin_A',
       'TM2Marin_B', 'TM2Marin_FT', 'TM2Marin_LANES', 'TM2Marin_ASSIGNABLE',
       'TM2_A', 'TM2_B', 'TM2_FT', 'TM2_FT_def', 'TM2_LANES', 'TM2_ASSIGNABLE',
       'sfcta_A', 'sfcta_B', 'sfcta_STREETNAME', 'sfcta_FT', 'sfcta_LANE_AM',
       'sfcta_LANE_OP', 'sfcta_LANE_PM', 'PEMSID', 'pems_lanes_FF',
       'pems_lanes_FR', 'pems_lanes_HV', 'pems_lanes_ML', 'pems_lanes_OR'],
      dtype='object')

In [6]:
link_df = pd.merge(link_df,
                  conflation_df[["shstReferenceId", 'tomtom_shieldnum', 'tomtom_rtedir']],
                  how = "left",
                  on = 'shstReferenceId')

In [7]:
link_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 1705772 entries, 0 to 1705771
Data columns (total 35 columns):
access                object
area                  object
bike_access           int64
bridge                object
drive_access          int64
est_width             object
fromIntersectionId    object
highway               object
id_x                  object
junction              object
key                   object
landuse               object
lanes                 object
link                  object
maxspeed              object
name                  object
nodeIds               object
oneWay                object
ref                   object
roadway               object
roundabout            object
service               object
shstGeometryId        object
shstReferenceId       object
toIntersectionId      object
tunnel                object
u                     int64
v                     int64
walk_access           int64
wayId                 object
width         

In [8]:
pems_file = "../../data/external/mtc/pems_period.csv"

pems_df = pd.read_csv(pems_file)

In [9]:
pems_df["geometry"] = [Point(xy) for xy in zip(pems_df.longitude, pems_df.latitude)]

pems_gdf = gpd.GeoDataFrame(pems_df, geometry = pems_df["geometry"],
                            crs={"init" : "epsg:4326"})

pems_gdf = pems_gdf[~((pems_gdf.longitude.isnull()) | (pems_gdf.latitude.isnull()))]

# Write out pems for shst conflation

In [ ]:
# keep unique

pems_gdf.drop_duplicates(subset = ["station", "longitude", "latitude"])[["station", "longitude", "latitude", "geometry"]].to_file("../../data/external/mtc/pems.in.geojson",
                                                driver = "GeoJSON")

# Prepare for nearest match

In [10]:
# convert crs to meter based

pems_gdf = pems_gdf.to_crs(epsg = 26915)

link_df = link_df.to_crs(epsg = 26915)

In [11]:
pems_gdf.shape

(76057, 23)

In [12]:
pems_gdf.route.value_counts().sum()

76057

In [13]:
link_df.sindex

In [14]:
pems_gdf.iloc[2]

station                                               400000
district                                                   4
route                                                    101
direction                                                  S
type                                                      ML
time_period                                               EV
lanes                                                      3
median_flow                                             8523
avg_flow                                             8434.26
sd_flow                                              420.136
median_speed                                         67.3375
avg_speed                                            67.2283
sd_speed                                             0.69445
median_occup                                        0.026681
avg_occup                                           0.026709
sd_occupancy                                      0.00136179
days_observed           

In [15]:
link_df[link_df.tomtom_shieldnum == "101"][["tomtom_rtedir", "roadway"]]

,tomtom_rtedir,roadway
4970,S,motorway
6024,N,motorway
6125,S,motorway
6956,S,motorway
6968,S,motorway
8399,,primary
9141,N,motorway
9512,N,motorway
9548,,primary
10390,N,motorway


In [16]:
link_df[link_df.tomtom_shieldnum == "101"].roadway.value_counts()

motorway         570
primary          180
motorway_link     33
trunk             16
secondary          4
residential        2
primary_link       1
Name: roadway, dtype: int64

# Write out links that have shieldnum same as pems route

In [17]:
[c for c in pems_gdf.route.unique().astype(str) if c not in link_df.tomtom_shieldnum.unique()]

['948']

In [18]:
link_df.tomtom_shieldnum.unique()
pems_gdf.route.unique().astype(str)

array(['101', '80', '680', '280', '880', '580', '24', '4', '238', '85',
       '87', '17', '237', '242', '92', '1', '980', '25', '37', '948',
       '780', '84', '29', '156', '205', '380', '12', '152', '160'],
      dtype='<U21')

In [19]:
interest_facility_df = link_df[link_df.tomtom_shieldnum.isin(pems_gdf.route.unique().astype(str))]

In [20]:
interest_facility_df = interest_facility_df.to_crs(epsg = 4326)

In [21]:
interest_facility_df[
    ["roadway", "shstReferenceId", "geometry"]].to_file(data_interim_dir + "mtc/link_candidates_for_pems.geojson",
                            driver = "GeoJSON")

PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'd:\\github\\lfs-cleaning\\travel-model-two-networks\\data\\interim\\mtc\\link_candidates_for_pems.geojson'

In [22]:
interest_facility_df.groupby(["tomtom_shieldnum", "tomtom_rtedir"]).count()

access  area  bike_access  bridge  \
tomtom_shieldnum tomtom_rtedir                                      
1                                 1205  1205         1205    1205   
                 N                 121   121          121     121   
                 S                 140   140          140     140   
101                                205   205          205     205   
                 N                 284   284          284     284   
                 S                 317   317          317     317   
12                                1007  1007         1007    1007   
                 E                  17    17           17      17   
                 W                  18    18           18      18   
152                                388   388          388     388   
156                                  4     4            4       4   
160                                  2     2            2       2   
                 N                   4     4            4       4   
                 S                   3     3            3       3   
17                                  27    27           27      27   
                 N                  17    17           17      17   
                 S                  17    17           17      17   
205              E                   1     1            1       1   
                 W                   2     2            2       2   
237                                 82    82           82      82   
                 E                  20    20           20      20   
                 W                  20    20           20      20   
238                                498   498          498     498   
                 N                  10    10           10      10   
                 S                   6     6            6       6   
24               E                  31    31           31      31   
                 W                  29    29           29      29   
242              N                   7     7            7       7   
                 S                   8     8            8       8   
25                                  20    20           20      20   
...                                ...   ...          ...     ...   
37               E                  13    13           13      13   
                 W                  21    21           21      21   
380              E                   5     5            5       5   
                 W                   4     4            4       4   
4                                   96    96           96      96   
                 E                  57    57           57      57   
                 W                  57    57           57      57   
580              E                 134   134          134     134   
                 W                 118   118          118     118   
680              N                 110   110          110     110   
                 S                 108   108          108     108   
780              E                  19    19           19      19   
                 W                  19    19           19      19   
80               E                 127   127          127     127   
                 W                 145   145          145     145   
84                                 818   818          818     818   
                 E                  12    12           12      12   
                 W                  19    19           19      19   
85               N                  56    56           56      56   
                 S                  46    46           46      46   
87                                  11    11           11      11   
                 N                  17    17           17      17   
                 S                  23    23           23      23   
880              N                 106   106          106     106   
                 S                 105   105          105     105   
92                            

In [23]:
pems_gdf.groupby(["route", "direction"]).count()

station  district  type  time_period  lanes  median_flow  \
route direction                                                             
1     N               35        35    35           35     35           35   
      S               30        30    30           30     30           30   
4     E             1756      1756  1756         1756   1756         1756   
      W             1800      1800  1800         1800   1800         1800   
12    E              110       110   110          110    110          110   
      W              100       100   100          100    100          100   
17    N              700       700   700          700    700          700   
      S              439       439   439          439    439          439   
24    E             1221      1221  1221         1221   1221         1221   
      W             1455      1455  1455         1455   1455         1455   
25    N              182       182   182          182    182          182   
      S              183       183   183          183    183          183   
29    N               65        65    65           65     65           65   
      S               75        75    75           75     75           75   
37    E              283       283   283          283    283          283   
      W              352       352   352          352    352          352   
80    E             5167      5167  5167         5167   5167         5167   
      W             5089      5089  5089         5089   5089         5089   
84    E              355       355   355          355    355          355   
      W              330       330   330          330    330          330   
85    N             1670      1670  1670         1670   1670         1670   
      S             1635      1635  1635         1635   1635         1635   
87    N              652       652   652          652    652          652   
      S              674       674   674          674    674          674   
92    E              727       727   727          727    727          727   
      W              729       729   729          729    729          729   
101   N             9914      9914  9914         9914   9914         9914   
      S             9344      9344  9344         9344   9344         9344   
152   E               30        30    30           30     30           30   
      W               25        25    25           25     25           25   
156   E               20        20    20           20     20           20   
      W               20        20    20           20     20           20   
160   N               15        15    15           15     15           15   
      S               15        15    15           15     15           15   
205   E               15        15    15           15     15           15   
      W               19        19    19           19     19           19   
237   E              388       388   388          388    388          388   
      W              509       509   509          509    509          509   
238   N              291       291   291          291    291          291   
      S              299       299   299          299    299          299   
242   N              224       224   224          224    224          224   
      S              232       232   232          232    232          232   
280   N             1832      1832  1832         1832   1832         1832   
      S             1794      1794  1794         1794   1794         1794   
380   E               20        20    20           20     20           20   
      W               20        20    20           20     20           20   
580   E             2765      2765  2765         2765   2765         2765   
      W             2671      2671  2671         2671   2671         2671   
680   N             4358      4358  4358         4358   4358         4358   
      S             4687      4687  4687         4687   4687         4687   
780   E      

In [24]:
link_df.tomtom_shieldnum.value_counts()

        862208
82        2703
1         1466
12        1042
84         849
101        806
185        768
123        739
116        716
128        611
238        514
35         513
29         507
152        388
130        387
G4         317
121        294
G2         278
80         272
580        252
J2         251
280        251
61         242
92         235
680        218
113        214
880        211
4          210
13         209
9          189
         ...  
131        125
237        122
G3         106
85         102
112         88
260         77
2063        74
37          73
G8          64
10          62
17          61
24          60
109         59
87          51
780         38
221         29
220         27
G9          27
93          24
25          20
505         19
262         19
242         15
980         14
G7          12
77          12
160          9
380          9
156          4
205          3
Name: tomtom_shieldnum, Length: 61, dtype: int64

In [25]:
pems_gdf[(pems_gdf.route == 101) & (pems_gdf.direction == "S")]

,station,district,route,direction,type,time_period,lanes,median_flow,avg_flow,sd_flow,...,median_occup,avg_occup,sd_occupancy,days_observed,state_pm,abs_pm,latitude,longitude,year,geometry
0,400000,4,101,S,ML,AM,3,16383.0,16237.843137,691.559263,...,0.130709,0.141076,0.047421,51,288.0,459.520,38.081167,-122.547606,2005,POINT (-2115315.56035313 4652743.598747026)
1,400000,4,101,S,ML,EA,3,4366.5,4376.357143,102.251993,...,0.041496,0.041541,0.001666,56,288.0,459.520,38.081167,-122.547606,2005,POINT (-2115315.56035313 4652743.598747026)
2,400000,4,101,S,ML,EV,3,8523.0,8434.255814,420.135754,...,0.026681,0.026709,0.001362,43,288.0,459.520,38.081167,-122.547606,2005,POINT (-2115315.56035313 4652743.598747026)
3,400000,4,101,S,ML,MD,3,18178.0,18298.847826,560.210197,...,0.067676,0.068619,0.004611,46,288.0,459.520,38.081167,-122.547606,2005,POINT (-2115315.56035313 4652743.598747026)
4,400000,4,101,S,ML,PM,3,14616.5,14651.839286,471.933769,...,0.065185,0.065373,0.002153,56,288.0,459.520,38.081167,-122.547606,2005,POINT (-2115315.56035313 4652743.598747026)
66,400043,4,101,S,ML,AM,4,23577.0,23416.629630,748.084530,...,0.098076,0.103209,0.014871,54,445.0,431.631,37.753591,-122.403092,2005,POINT (-2115244.798598309 4610955.231681113)
67,400043,4,101,S,ML,EA,4,4928.0,4930.222222,173.380659,...,0.033719,0.033801,0.001366,63,445.0,431.631,37.753591,-122.403092,2005,POINT (-2115244.798598309 4610955.231681113)
68,400043,4,101,S,ML,EV,4,25855.5,25716.104167,1250.793778,...,0.067001,0.067381,0.003901,48,445.0,431.631,37.753591,-122.403092,2005,POINT (-2115244.798598309 4610955.231681113)
69,400043,4,101,S,ML,MD,4,28495.0,28507.380000,651.071264,...,0.095326,0.095746,0.002916,50,445.0,431.631,37.753591,-122.403092,2005,POINT (-2115244.798598309 4610955.231681113)
70,400043,4,101,S,ML,PM,4,26542.0,26266.150000,914.049248,...,0.118074,0.126354,0.019513,60,445.0,431.631,37.753591,-122.403092,2005,POINT (-2115244.798598309 4610955.231681113)


In [26]:
pems_gdf[(pems_gdf.route == 101) & (pems_gdf.direction == "S")]["type"].value_counts()

ML    7112
OR    1287
FR     885
FF      60
Name: type, dtype: int64

In [27]:
pems_gdf[(pems_gdf["type"] == "HV")]

,station,district,route,direction,type,time_period,lanes,median_flow,avg_flow,sd_flow,...,median_occup,avg_occup,sd_occupancy,days_observed,state_pm,abs_pm,latitude,longitude,year,geometry
59758,407332,4,85,N,HV,AM,1,3927.5,3898.166667,258.362514,...,0.084976,0.086073,0.011432,24,1439.0,23.886,37.407311,-122.069566,2015,POINT (-2098485.810211925 4561187.769523272)
59759,407332,4,85,N,HV,EA,1,474.5,472.041667,23.456675,...,0.014065,0.013993,0.001079,24,1439.0,23.886,37.407311,-122.069566,2015,POINT (-2098485.810211925 4561187.769523272)
59760,407332,4,85,N,HV,EV,1,1872.5,1862.409091,111.751280,...,0.026427,0.026462,0.001526,22,1439.0,23.886,37.407311,-122.069566,2015,POINT (-2098485.810211925 4561187.769523272)
59761,407332,4,85,N,HV,MD,1,4246.0,4265.304348,186.052152,...,0.059810,0.060197,0.003599,23,1439.0,23.886,37.407311,-122.069566,2015,POINT (-2098485.810211925 4561187.769523272)
59762,407332,4,85,N,HV,PM,1,1531.0,1511.695652,111.579745,...,0.026375,0.026147,0.002041,23,1439.0,23.886,37.407311,-122.069566,2015,POINT (-2098485.810211925 4561187.769523272)
59818,407359,4,85,S,HV,AM,1,1006.5,991.458333,58.596616,...,0.023288,0.023117,0.001375,24,1439.0,23.886,37.407147,-122.070055,2015,POINT (-2098536.5811206 4561184.103187212)
59819,407359,4,85,S,HV,EA,1,1.5,1.791667,1.559798,...,0.000100,0.000099,0.000086,24,1439.0,23.886,37.407147,-122.070055,2015,POINT (-2098536.5811206 4561184.103187212)
59820,407359,4,85,S,HV,EV,1,2763.5,2715.000000,262.268310,...,0.054162,0.053800,0.007297,22,1439.0,23.886,37.407147,-122.070055,2015,POINT (-2098536.5811206 4561184.103187212)
59821,407359,4,85,S,HV,MD,1,2970.0,3014.391304,226.936262,...,0.046906,0.047906,0.004180,23,1439.0,23.886,37.407147,-122.070055,2015,POINT (-2098536.5811206 4561184.103187212)
59822,407359,4,85,S,HV,PM,1,3095.0,3111.695652,230.054625,...,0.061583,0.063031,0.006688,23,1439.0,23.886,37.407147,-122.070055,2015,POINT (-2098536.5811206 4561184.103187212)


# Match Pems to the nearest link that has same shieldnum and direction

In [28]:
pems_gdf["type"].value_counts()

ML    60319
OR     9309
FR     5733
FF      666
HV       30
Name: type, dtype: int64

In [29]:
# dictionary for pems type and roadway type
roadway_type_list = link_df.roadway.unique()

pems_type_list = pems_gdf["type"].unique()

pems_roadway_crosswalk = {"ML" : ['tertiary', 'primary', 'secondary', 'motorway', 'trunk'],
                          "HV" : [c for c in roadway_type_list if c.endswith("_link")],
                          "FF" : [c for c in roadway_type_list if c.endswith("_link")],
                          "OR" : [c for c in roadway_type_list if c.endswith("_link")],
                          "FR" : [c for c in roadway_type_list if c.endswith("_link")]
                         }

In [54]:
# match based on pems route, direction, and tomtom shieldnum, rtedir

offset = 100

pems_route_list = pems_gdf.route.unique().tolist()

pems_match_gdf = gpd.GeoDataFrame()

for route in pems_route_list:
#for route in [80]:
    
    print("pems route id {}".format(route))
    
    pems_subset_gdf = pems_gdf[(pems_gdf.route == route)].copy()
    
    dir_list = pems_subset_gdf.direction.unique().tolist()
    
    for direction in dir_list:
    #for direction in ["W"]:
        
        print("\t pems direction {}".format(direction))
        
        pems_subset_gdf = pems_gdf[(pems_gdf.route == route) & 
                                   (pems_gdf.direction == direction)].copy()
        
        type_list = pems_subset_gdf["type"].unique().tolist()
        
        for ptype in type_list:
        #for ptype in ["ML"]:
            
            print("\t\t pems type {}".format(ptype))
            
            pems_subset_gdf = pems_gdf[(pems_gdf.route == route) &
                                       (pems_gdf.direction == direction) &
                                       (pems_gdf["type"] == ptype)].copy()
            
            bbox = pems_subset_gdf.bounds + [-offset, -offset, offset, offset]
    
            line = link_df[(link_df.tomtom_shieldnum == str(route)) & 
                           (link_df.tomtom_rtedir == direction) &
                           (link_df.roadway.isin(pems_roadway_crosswalk[ptype])
                           )].copy()
            
            if len(line) == 0:
                print("\t\t there is no link with tomtom label direction {}, route {}, roadway {}, matching to the closest\
                      {}".format(
                    direction, route, ptype, pems_roadway_crosswalk[ptype]
                ))
                
                line = link_df[(link_df.roadway.isin(pems_roadway_crosswalk[ptype]))
                              ].copy()
        
            hits = bbox.apply(lambda row: list(line.sindex.intersection(row)),
                  axis = 1)
            
            tmp = pd.DataFrame({
                # index of points table
                "pt_idx": np.repeat(hits.index, hits.apply(len)),
                # ordinal position of line - access via iloc later
                "line_i": np.concatenate(hits.values)
            })
        
            # join with pems
        
            tmp.set_index(["pt_idx"], inplace = True)
        
            tmp = tmp.join(pems_subset_gdf[["station", "longitude", "latitude", "route", "direction", "type","geometry"]
                                          ].rename(
                                columns = {"geometry" : "point"}), 
                           how = "left")
            
            # join with links
        
            tmp.set_index(["line_i"], inplace = True)
        
            tmp = tmp.join(line[["shstReferenceId", "roadway","tomtom_shieldnum", "tomtom_rtedir", "geometry"]
                           ].reset_index(drop=True), 
                       how="left")
        
            # find closest line to point
        
            tmp = gpd.GeoDataFrame(tmp, geometry = tmp["geometry"], crs = pems_gdf.crs)
        
            tmp["snap_distance"]  = tmp.geometry.distance(gpd.GeoSeries(tmp.point))
        
            tmp.sort_values(by = ["snap_distance"], inplace = True)
        
            closest = tmp.groupby(["station", "longitude", "latitude"]).first().reset_index()
        
            pems_match_gdf = pd.concat([pems_match_gdf, closest],
                                       sort = False,
                                       ignore_index = True)

pems route id 101
	 pems direction S
		 pems type ML
		 pems type FR
		 pems type OR
		 pems type FF
	 pems direction N
		 pems type ML
		 pems type FR
		 pems type OR
		 pems type FF
pems route id 80
	 pems direction W
		 pems type ML
		 pems type FR
		 pems type FF
		 pems type OR
	 pems direction E
		 pems type ML
		 pems type FF
		 pems type OR
		 pems type FR
pems route id 680
	 pems direction N
		 pems type ML
		 pems type OR
		 there is no link with tomtom label direction N, route 680, roadway OR, matching to the closest                      ['primary_link', 'motorway_link', 'trunk_link', 'secondary_link', 'tertiary_link']
		 pems type FR
		 there is no link with tomtom label direction N, route 680, roadway FR, matching to the closest                      ['primary_link', 'motorway_link', 'trunk_link', 'secondary_link', 'tertiary_link']
		 pems type FF
		 there is no link with tomtom label direction N, route 680, roadway FF, matching to the closest                      ['primary

pems route id 780
	 pems direction E
		 pems type ML
	 pems direction W
		 pems type ML
pems route id 84
	 pems direction E
		 pems type ML
	 pems direction W
		 pems type ML
pems route id 29
	 pems direction N
		 pems type ML
		 pems type FR
		 pems type OR
	 pems direction S
		 pems type ML
		 pems type FR
		 pems type OR
pems route id 156
	 pems direction W
		 pems type ML
		 there is no link with tomtom label direction W, route 156, roadway ML, matching to the closest                      ['tertiary', 'primary', 'secondary', 'motorway', 'trunk']
	 pems direction E
		 pems type ML
		 there is no link with tomtom label direction E, route 156, roadway ML, matching to the closest                      ['tertiary', 'primary', 'secondary', 'motorway', 'trunk']
pems route id 205
	 pems direction E
		 pems type ML
	 pems direction W
		 pems type ML
pems route id 380
	 pems direction E
		 pems type ML
	 pems direction W
		 pems type ML
pems route id 12
	 pems direction E
		 pems type ML
	 pe

In [55]:
pems_match_gdf

,station,longitude,latitude,route,direction,type,point,shstReferenceId,roadway,tomtom_shieldnum,tomtom_rtedir,geometry,snap_distance
0,400000.0,-122.547963,38.081498,101,S,ML,POINT (-2115334.47138468 4652792.447341145),df209cac09d05a2f7e0a22dc8e931df8,motorway,101,S,LINESTRING (-2115401.040652251 4652959.3807638...,2.867350
1,400000.0,-122.547606,38.081167,101,S,ML,POINT (-2115315.56035313 4652743.598747026),df209cac09d05a2f7e0a22dc8e931df8,motorway,101,S,LINESTRING (-2115401.040652251 4652959.3807638...,3.174880
2,400002.0,-122.328465,37.584097,101,S,ML,POINT (-2115131.410841958 4589330.976028608),dfd26b2508d008ad867b43ce2eae005f,motorway,101,S,LINESTRING (-2117606.566968283 4590672.7751965...,3.106433
3,400043.0,-122.403092,37.753591,101,S,ML,POINT (-2115244.798598309 4610955.231681113),6d8b2509ddc5a1a71b55ff5b569e136a,motorway,101,S,LINESTRING (-2114949.61504092 4612440.25763794...,2.475063
4,400069.0,-121.977493,37.385758,101,S,ML,POINT (-2090942.69189868 4555908.530250972),7dc48aa303c48fb6eea805245d5af370,motorway,101,S,LINESTRING (-2091431.901169152 4556300.2670397...,4.480192
5,400098.0,-122.359083,37.588702,101,S,ML,POINT (-2117726.824923707 4590807.944753751),78430402f0e1d13185c6afffd6adae54,motorway,101,S,LINESTRING (-2118058.793265421 4591315.5275253...,4.836421
6,400106.0,-122.332169,37.586325,101,S,ML,POINT (-2115380.128588265 4589699.913306671),dfd26b2508d008ad867b43ce2eae005f,motorway,101,S,LINESTRING (-2117606.566968283 4590672.7751965...,2.404988
7,400106.0,-122.330411,37.585602,101,S,ML,POINT (-2115249.019242223 4589562.902332258),dfd26b2508d008ad867b43ce2eae005f,motorway,101,S,LINESTRING (-2117606.566968283 4590672.7751965...,3.498507
8,400109.0,-121.889426,37.362706,101,S,ML,POINT (-2083815.400358296 4550591.954518915),b638c32df10b558df568f0ec0b5c5aa9,motorway,101,S,LINESTRING (-2084331.15084435 4550835.52599223...,2.448930
9,400116.0,-122.403213,37.634035,101,S,ML,POINT (-2119949.813978697 4597346.423843719),de5839cde1e16bbac638045817fb0fe3,motorway,101,S,LINESTRING (-2119948.934662843 4597881.8194357...,3.546147


In [56]:
pems_nearest_gdf = pd.merge(pems_gdf,
                            pems_match_gdf.drop(["point", "geometry"], axis = 1),
                            how = "left",
                            on = ["station", "longitude", "latitude", "route", "direction", "type"])

In [57]:
pems_gdf.shape

(76057, 23)

In [58]:
pems_nearest_gdf.shape

(76057, 28)

In [59]:
pems_match_gdf[pems_match_gdf.station == 400260]

,station,longitude,latitude,route,direction,type,point,shstReferenceId,roadway,tomtom_shieldnum,tomtom_rtedir,geometry,snap_distance
22,400260.0,-122.393983,37.703072,101,S,ML,POINT (-2116405.519119958 4604919.938460853),e0baa6ac9c09cf1bf3a86f61942ae2a0,motorway,101,S,LINESTRING (-2116265.063105721 4605784.1925672...,1.093781
384,400260.0,-122.394455,37.706585,101,N,ML,POINT (-2116310.310158523 4605334.623277964),b958a1373a6c1ce3a73a6452ad1ddb96,motorway,101,N,LINESTRING (-2116435.958658942 4604533.4837011...,1.892163
385,400260.0,-122.393983,37.703072,101,N,ML,POINT (-2116405.519119958 4604919.938460853),b958a1373a6c1ce3a73a6452ad1ddb96,motorway,101,N,LINESTRING (-2116435.958658942 4604533.4837011...,28.904669


In [60]:
pems_match_gdf[pems_match_gdf.station.isin([407358, 407359])]

,station,longitude,latitude,route,direction,type,point,shstReferenceId,roadway,tomtom_shieldnum,tomtom_rtedir,geometry,snap_distance


In [61]:
pems_gdf[pems_gdf.station == 407359]

,station,district,route,direction,type,time_period,lanes,median_flow,avg_flow,sd_flow,...,median_occup,avg_occup,sd_occupancy,days_observed,state_pm,abs_pm,latitude,longitude,year,geometry
59818,407359,4,85,S,HV,AM,1,1006.5,991.458333,58.596616,...,0.023288,0.023117,0.001375,24,1439.0,23.886,37.407147,-122.070055,2015,POINT (-2098536.5811206 4561184.103187212)
59819,407359,4,85,S,HV,EA,1,1.5,1.791667,1.559798,...,0.000100,0.000099,0.000086,24,1439.0,23.886,37.407147,-122.070055,2015,POINT (-2098536.5811206 4561184.103187212)
59820,407359,4,85,S,HV,EV,1,2763.5,2715.000000,262.268310,...,0.054162,0.053800,0.007297,22,1439.0,23.886,37.407147,-122.070055,2015,POINT (-2098536.5811206 4561184.103187212)
59821,407359,4,85,S,HV,MD,1,2970.0,3014.391304,226.936262,...,0.046906,0.047906,0.004180,23,1439.0,23.886,37.407147,-122.070055,2015,POINT (-2098536.5811206 4561184.103187212)
59822,407359,4,85,S,HV,PM,1,3095.0,3111.695652,230.054625,...,0.061583,0.063031,0.006688,23,1439.0,23.886,37.407147,-122.070055,2015,POINT (-2098536.5811206 4561184.103187212)
69413,407359,4,85,S,HV,AM,1,1064.0,1066.929577,78.441484,...,0.024396,0.024497,0.002095,71,1665.0,23.886,37.407147,-122.070055,2016,POINT (-2098536.5811206 4561184.103187212)
69414,407359,4,85,S,HV,EA,1,1.0,2.270270,2.400327,...,0.000100,0.000128,0.000150,74,1665.0,23.886,37.407147,-122.070055,2016,POINT (-2098536.5811206 4561184.103187212)
69415,407359,4,85,S,HV,EV,1,2906.0,2867.746479,275.940714,...,0.054315,0.054549,0.006617,71,1665.0,23.886,37.407147,-122.070055,2016,POINT (-2098536.5811206 4561184.103187212)
69416,407359,4,85,S,HV,MD,1,3110.0,3126.531250,266.418454,...,0.049394,0.049818,0.004694,64,1665.0,23.886,37.407147,-122.070055,2016,POINT (-2098536.5811206 4561184.103187212)
69417,407359,4,85,S,HV,PM,1,3218.5,3197.764706,206.223840,...,0.064881,0.067903,0.011518,68,1665.0,23.886,37.407147,-122.070055,2016,POINT (-2098536.5811206 4561184.103187212)


In [62]:
pems_nearest_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 76057 entries, 0 to 76056
Data columns (total 28 columns):
station             76057 non-null int64
district            76057 non-null int64
route               76057 non-null int64
direction           76057 non-null object
type                76057 non-null object
time_period         76057 non-null object
lanes               76057 non-null int64
median_flow         76057 non-null float64
avg_flow            76057 non-null float64
sd_flow             76057 non-null float64
median_speed        60350 non-null float64
avg_speed           60350 non-null float64
sd_speed            60350 non-null float64
median_occup        76057 non-null float64
avg_occup           76057 non-null float64
sd_occupancy        76057 non-null float64
days_observed       76057 non-null int64
state_pm            76057 non-null float64
abs_pm              76057 non-null float64
latitude            76057 non-null float64
longitude           76057 non-null f

In [63]:
pems_nearest_gdf[pems_nearest_gdf.shstReferenceId.notnull()].station.nunique()

2440

In [64]:
pems_nearest_gdf[pems_nearest_gdf.shstReferenceId.isnull()].station.nunique()

1286

In [65]:
pems_nearest_gdf[pems_nearest_gdf.shstReferenceId.notnull()]["type"].value_counts()

ML    54576
OR     1476
FR      658
FF      201
HV       15
Name: type, dtype: int64

In [66]:
pems_nearest_gdf[(pems_nearest_gdf.station == 400009)]

,station,district,route,direction,type,time_period,lanes,median_flow,avg_flow,sd_flow,...,abs_pm,latitude,longitude,year,geometry,shstReferenceId,roadway,tomtom_shieldnum,tomtom_rtedir,snap_distance
15,400009,4,80,W,ML,AM,5,16085.0,15930.111111,577.366524,...,10.94,37.863905,-122.303083,2005,POINT (-2101884.901027649 4620392.963378702),NaN,NaN,NaN,NaN,NaN
16,400009,4,80,W,ML,EA,5,4374.0,4376.000000,95.348801,...,10.94,37.863905,-122.303083,2005,POINT (-2101884.901027649 4620392.963378702),NaN,NaN,NaN,NaN,NaN
17,400009,4,80,W,ML,EV,5,10881.0,11121.137931,2305.214349,...,10.94,37.863905,-122.303083,2005,POINT (-2101884.901027649 4620392.963378702),NaN,NaN,NaN,NaN,NaN
18,400009,4,80,W,ML,MD,5,17734.0,17643.592593,953.089319,...,10.94,37.863905,-122.303083,2005,POINT (-2101884.901027649 4620392.963378702),NaN,NaN,NaN,NaN,NaN
19,400009,4,80,W,ML,PM,5,12425.0,12358.562500,972.879893,...,10.94,37.863905,-122.303083,2005,POINT (-2101884.901027649 4620392.963378702),NaN,NaN,NaN,NaN,NaN
3150,400009,4,80,W,ML,AM,5,10880.0,12342.600000,2987.872467,...,10.94,37.863905,-122.303083,2006,POINT (-2101884.901027649 4620392.963378702),NaN,NaN,NaN,NaN,NaN
3151,400009,4,80,W,ML,EA,5,3040.5,3517.119048,774.182313,...,10.94,37.863905,-122.303083,2006,POINT (-2101884.901027649 4620392.963378702),NaN,NaN,NaN,NaN,NaN
3152,400009,4,80,W,ML,EV,5,7167.5,8314.100000,2461.927611,...,10.94,37.863905,-122.303083,2006,POINT (-2101884.901027649 4620392.963378702),NaN,NaN,NaN,NaN,NaN
3153,400009,4,80,W,ML,MD,5,11704.0,13646.395349,2917.419058,...,10.94,37.863905,-122.303083,2006,POINT (-2101884.901027649 4620392.963378702),NaN,NaN,NaN,NaN,NaN
3154,400009,4,80,W,ML,PM,5,8260.0,9479.568627,2047.528815,...,10.94,37.863905,-122.303083,2006,POINT (-2101884.901027649 4620392.963378702),NaN,NaN,NaN,NaN,NaN


# Add shst conflation

In [74]:
pems_shst_match_df = read_shst_extract(data_interim_dir + "mtc/", "*pems.out.matched.geojson")

pems_shst_match_df.rename(columns = {"shstFromIntersectionId" : "fromIntersectionId",
                                   "shstToIntersectionId" : "toIntersectionId"},
                       inplace = True)

----------start reading shst extraction data-------------
reading shst extraction data :  ../../data/interim/mtc\car_rules\pems.out.matched.geojson
----------finished reading shst extraction data-------------


In [75]:
pems_shst_match_df.rename(columns = {"pp_station" : "station", "pp_longitude" : "longitude", "pp_latitude" : "latitude",
                                     "referenceId" : "shstReferenceId"},
                          inplace = True)

In [79]:
pems_shst_match_df[pems_shst_match_df["station"] == 407359]

,score,location,referenceLength,geometryId,shstReferenceId,direction,bearing,sideOfStreet,interceptAngle,station,longitude,latitude,geometry,source
2979,1.930479,420.632993,698.11,8683632a86788660cd1453f3cf7a63d8,00ff1d8493ea56e766bedbd47663cbb0,forward,165.600783,left,269.999833,407359,-122.070055,37.407147,POINT (-122.070076169564 37.40714268262898),../../data/interim/mtc\car_rules\pems.out.matc...


In [52]:
# if pems is not matched in previous method, use shst match

In [69]:
pems_nearest_not_matched_df = pems_nearest_gdf[pems_nearest_gdf.shstReferenceId.isnull()].copy()

In [70]:
pems_nearest_not_matched_df

,station,district,route,direction,type,time_period,lanes,median_flow,avg_flow,sd_flow,...,abs_pm,latitude,longitude,year,geometry,shstReferenceId,roadway,tomtom_shieldnum,tomtom_rtedir,snap_distance
15,400009,4,80,W,ML,AM,5,16085.0,15930.111111,577.366524,...,10.940,37.863905,-122.303083,2005,POINT (-2101884.901027649 4620392.963378702),NaN,NaN,NaN,NaN,NaN
16,400009,4,80,W,ML,EA,5,4374.0,4376.000000,95.348801,...,10.940,37.863905,-122.303083,2005,POINT (-2101884.901027649 4620392.963378702),NaN,NaN,NaN,NaN,NaN
17,400009,4,80,W,ML,EV,5,10881.0,11121.137931,2305.214349,...,10.940,37.863905,-122.303083,2005,POINT (-2101884.901027649 4620392.963378702),NaN,NaN,NaN,NaN,NaN
18,400009,4,80,W,ML,MD,5,17734.0,17643.592593,953.089319,...,10.940,37.863905,-122.303083,2005,POINT (-2101884.901027649 4620392.963378702),NaN,NaN,NaN,NaN,NaN
19,400009,4,80,W,ML,PM,5,12425.0,12358.562500,972.879893,...,10.940,37.863905,-122.303083,2005,POINT (-2101884.901027649 4620392.963378702),NaN,NaN,NaN,NaN,NaN
106,400060,4,80,W,ML,AM,5,11553.0,11468.783784,404.328946,...,11.450,37.870977,-122.305289,2005,POINT (-2101806.466966349 4621266.028712609),NaN,NaN,NaN,NaN,NaN
107,400060,4,80,W,ML,EA,5,1876.0,1874.604651,91.501012,...,11.450,37.870977,-122.305289,2005,POINT (-2101806.466966349 4621266.028712609),NaN,NaN,NaN,NaN,NaN
108,400060,4,80,W,ML,EV,5,15009.0,15072.648649,1057.086831,...,11.450,37.870977,-122.305289,2005,POINT (-2101806.466966349 4621266.028712609),NaN,NaN,NaN,NaN,NaN
109,400060,4,80,W,ML,MD,5,17398.0,17044.432432,1494.305218,...,11.450,37.870977,-122.305289,2005,POINT (-2101806.466966349 4621266.028712609),NaN,NaN,NaN,NaN,NaN
110,400060,4,80,W,ML,PM,5,15896.0,15203.186047,1968.801185,...,11.450,37.870977,-122.305289,2005,POINT (-2101806.466966349 4621266.028712609),NaN,NaN,NaN,NaN,NaN


In [126]:
def geodesic_point_buffer(lat, lon, meters):
    # Azimuthal equidistant projection
    aeqd_proj = '+proj=aeqd +lat_0={lat} +lon_0={lon} +x_0=0 +y_0=0'
    project = partial(
        pyproj.transform,
        pyproj.Proj(aeqd_proj.format(lat=lat, lon=lon)),
        proj_wgs84)
    buf = Point(0, 0).buffer(meters)  # distance in metres
    return Polygon(transform(project, buf).exterior.coords[:])

def links_within_stop_buffer(drive_link_df, stop, buffer_radius = 25):
    """
    find the links that are within buffer of nodes
    """
    
    stop_buffer_df = stop.copy()
    stop_buffer_df["geometry"] = stop_buffer_df.apply(lambda x: geodesic_point_buffer(x.latitude, x.longitude, buffer_radius), 
                                                      axis = 1)
    
    stop_buffer_df = gpd.GeoDataFrame(stop_buffer_df, geometry = stop_buffer_df["geometry"], crs = {'init' : 'epsg:4326'})
    
    stop_buffer_link_df = gpd.sjoin(drive_link_df, stop_buffer_df[["geometry", "type"]], 
                                             how = "left", op = "intersects")
    
    stop_buffer_link_df = stop_buffer_link_df[stop_buffer_link_df["type"].notnull()]
    
    return stop_buffer_link_df["shstReferenceId"].tolist()

In [72]:
pems_nearest_not_matched_df.head(3)

,station,district,route,direction,type,time_period,lanes,median_flow,avg_flow,sd_flow,...,abs_pm,latitude,longitude,year,geometry,shstReferenceId,roadway,tomtom_shieldnum,tomtom_rtedir,snap_distance
15,400009,4,80,W,ML,AM,5,16085.0,15930.111111,577.366524,...,10.94,37.863905,-122.303083,2005,POINT (-2101884.901027649 4620392.963378702),NaN,NaN,NaN,NaN,NaN
16,400009,4,80,W,ML,EA,5,4374.0,4376.000000,95.348801,...,10.94,37.863905,-122.303083,2005,POINT (-2101884.901027649 4620392.963378702),NaN,NaN,NaN,NaN,NaN
17,400009,4,80,W,ML,EV,5,10881.0,11121.137931,2305.214349,...,10.94,37.863905,-122.303083,2005,POINT (-2101884.901027649 4620392.963378702),NaN,NaN,NaN,NaN,NaN


In [151]:
from functools import partial
import pyproj
from shapely.ops import transform
from shapely.geometry import Point, Polygon

proj_wgs84 = pyproj.Proj('+proj=longlat +datum=WGS84')

offset = 100

unique_pems_nearest_not_matched_df = pems_nearest_not_matched_df.drop_duplicates(
    subset = ["station", "type", "latitude", "longitude"]
)

link_df = link_df.to_crs({'init': 'epsg:4326'})

In [161]:
import warnings
warnings.filterwarnings('ignore')

In [167]:
%%time

# match based on facility type

pems_matched_ft_gdf = gpd.GeoDataFrame()

for i in range(len(unique_pems_nearest_not_matched_df)):
    
    row_df = unique_pems_nearest_not_matched_df.iloc[[i]][["station", "type", "latitude", "longitude"]].copy()
    
    print("process station {}, type {}".format(row_df["station"].iloc[0], row_df["type"].iloc[0]))
    
    link = link_df[(link_df.roadway.isin(pems_roadway_crosswalk[row_df["type"].iloc[0]]))].copy()
    
    links_within_buffer = links_within_stop_buffer(link, row_df, offset)
    
    row_df["geometry"] = row_df.apply(lambda x: Point(x.longitude, x.latitude), axis = 1)
    
    # find closest line to point
    
    links_within_buffer_gdf = link_df[link_df.shstReferenceId.isin(links_within_buffer)].copy()
    
    links_within_buffer_gdf["station"] = row_df["station"].iloc[0]
    links_within_buffer_gdf["type"] = row_df["type"].iloc[0]
    links_within_buffer_gdf["latitude"] = row_df["latitude"].iloc[0]
    links_within_buffer_gdf["longitude"] = row_df["longitude"].iloc[0]
    links_within_buffer_gdf["point"] = row_df["geometry"].iloc[0]
        
    links_within_buffer_gdf["snap_distance"] = links_within_buffer_gdf.geometry.distance(
        gpd.GeoSeries(links_within_buffer_gdf.point)
    )
        
    links_within_buffer_gdf.sort_values(by = ["snap_distance"], inplace = True)
        
    closest_link_gdf = links_within_buffer_gdf.groupby(["station", "longitude", "latitude"]).first().reset_index()
    
    pems_matched_ft_gdf = pd.concat([pems_matched_ft_gdf, closest_link_gdf], sort = False, ignore_index = True)

process station 400009, type ML
process station 400060, type ML
process station 400108, type ML
process station 400126, type ML
process station 400170, type ML
process station 400176, type ML
process station 400214, type ML
process station 400242, type ML
process station 400280, type ML
process station 400283, type ML
process station 400340, type ML
process station 400349, type ML
process station 400367, type ML
process station 400403, type ML
process station 400432, type ML
process station 400433, type FF
process station 400467, type FF
process station 400553, type ML
process station 400612, type ML
process station 400679, type ML
process station 400691, type ML
process station 400696, type ML
process station 400704, type ML
process station 400712, type ML
process station 400728, type ML
process station 400749, type ML
process station 400768, type ML
process station 400803, type ML
process station 400808, type ML
process station 400923, type ML
process station 400962, type ML
process 

process station 402954, type OR
process station 402955, type OR
process station 402956, type FR
process station 402957, type OR
process station 402959, type OR
process station 402960, type OR
process station 402961, type OR
process station 402962, type FR
process station 402963, type OR
process station 402964, type OR
process station 402965, type OR
process station 402966, type OR
process station 402967, type FR
process station 402970, type OR
process station 402971, type OR
process station 402974, type FR
process station 402975, type OR
process station 402976, type OR
process station 402977, type OR
process station 402978, type FR
process station 402979, type OR
process station 402980, type FR
process station 402982, type OR
process station 402984, type OR
process station 402989, type OR
process station 402990, type OR
process station 402991, type FR
process station 402993, type OR
process station 402994, type OR
process station 402995, type OR
process station 402996, type OR
process 

process station 403182, type FR
process station 403184, type OR
process station 403185, type FR
process station 403187, type OR
process station 403189, type FR
process station 403214, type FF
process station 403239, type OR
process station 403300, type OR
process station 403322, type OR
process station 403330, type FR
process station 403331, type OR
process station 403339, type OR
process station 403341, type FR
process station 403342, type OR
process station 403344, type FR
process station 403345, type OR
process station 403347, type FR
process station 403348, type OR
process station 403350, type OR
process station 403358, type ML
process station 403359, type ML
process station 403360, type ML
process station 403394, type FR
process station 403410, type OR
process station 403411, type FR
process station 403413, type OR
process station 403415, type FR
process station 403416, type FR
process station 403454, type OR
process station 403998, type ML
process station 404291, type ML
process 

process station 407340, type FR
process station 407349, type OR
process station 407351, type FR
process station 407353, type OR
process station 407358, type FF
process station 407359, type HV
process station 407363, type FR
process station 407365, type OR
process station 407368, type OR
process station 407745, type FF
process station 407746, type FR
process station 407749, type OR
process station 407863, type ML
process station 407864, type FR
process station 407867, type OR
process station 407868, type FR
process station 407870, type OR
process station 407882, type ML
process station 407948, type FR
process station 407949, type OR
process station 407953, type OR
process station 407954, type OR
process station 407955, type FR
process station 407980, type FR
process station 407981, type OR
process station 407991, type OR
process station 407993, type FR
process station 407994, type OR
process station 408004, type FR
process station 408005, type OR
process station 408006, type FR
process 

process station 409937, type OR
process station 409939, type OR
process station 409944, type OR
process station 409945, type FR
process station 409947, type FR
process station 409957, type FR
process station 409967, type OR
process station 409999, type OR
process station 410000, type FR
process station 410002, type FR
process station 410085, type OR
process station 410086, type FR
process station 410087, type OR
process station 410094, type FR
process station 410095, type OR
process station 410101, type OR
process station 410103, type FR
process station 410107, type OR
process station 410109, type FR
process station 410110, type OR
process station 410111, type FR
process station 410113, type OR
process station 410115, type OR
process station 410116, type FR
process station 410141, type FR
process station 410144, type FR
process station 410145, type OR
process station 410160, type OR
process station 410164, type FR
process station 410166, type OR
process station 410211, type OR
process 

process station 412440, type OR
process station 412441, type FR
process station 412442, type OR
process station 412445, type FR
process station 413037, type OR
process station 413038, type FR
process station 413206, type OR
process station 413819, type OR
process station 413845, type ML
process station 413869, type FR
process station 413879, type OR
process station 413880, type FR
process station 414029, type OR
process station 414030, type FR
process station 414045, type OR
process station 414304, type OR
process station 414310, type OR
process station 414478, type OR
process station 414479, type FR
process station 414488, type OR
process station 414695, type FR
process station 414696, type OR
process station 414706, type OR
process station 414796, type FR
process station 414797, type FR
process station 414798, type OR
process station 414799, type OR
process station 414820, type FR
process station 414821, type OR
process station 414822, type FR
process station 414918, type FR
process 

In [169]:
pems_use_ft_result_df = pd.merge(pems_nearest_not_matched_df.drop("shstReferenceId", axis = 1),
                                   pems_matched_ft_gdf[["shstReferenceId", 'station', 'longitude', 'latitude', 'type']],
                                   how = "left",
                                   on = ['station', 'type','longitude', 'latitude'])

In [181]:
pems_use_ft_result_df[(pems_use_ft_result_df.shstReferenceId.isnull()) & (pems_use_ft_result_df["type"] == "ML")]

,station,district,route,direction,type,time_period,lanes,median_flow,avg_flow,sd_flow,...,abs_pm,latitude,longitude,year,geometry,roadway,tomtom_shieldnum,tomtom_rtedir,snap_distance,shstReferenceId
4429,403471,4,17,N,ML,AM,2,8293.0,7839.237288,1056.806728,...,5.45,37.061130,-122.004466,2013,POINT (-2105821.363014829 4519711.59987066),NaN,NaN,NaN,NaN,NaN
4430,403471,4,17,N,ML,EA,2,1618.0,1604.821429,63.596560,...,5.45,37.061130,-122.004466,2013,POINT (-2105821.363014829 4519711.59987066),NaN,NaN,NaN,NaN,NaN
4431,403471,4,17,N,ML,EV,2,2773.5,2720.727273,460.612344,...,5.45,37.061130,-122.004466,2013,POINT (-2105821.363014829 4519711.59987066),NaN,NaN,NaN,NaN,NaN
4432,403471,4,17,N,ML,MD,2,6175.0,6005.883333,883.559060,...,5.45,37.061130,-122.004466,2013,POINT (-2105821.363014829 4519711.59987066),NaN,NaN,NaN,NaN,NaN
4433,403471,4,17,N,ML,PM,2,5133.0,5090.069767,662.887385,...,5.45,37.061130,-122.004466,2013,POINT (-2105821.363014829 4519711.59987066),NaN,NaN,NaN,NaN,NaN
4434,403475,4,17,S,ML,AM,2,3395.0,3221.186441,501.359066,...,5.45,37.061194,-122.004672,2013,POINT (-2105837.76253553 4519725.204552194),NaN,NaN,NaN,NaN,NaN
4435,403475,4,17,S,ML,EA,2,213.0,198.051724,93.424784,...,5.45,37.061194,-122.004672,2013,POINT (-2105837.76253553 4519725.204552194),NaN,NaN,NaN,NaN,NaN
4436,403475,4,17,S,ML,EV,2,5174.5,5208.636364,373.960852,...,5.45,37.061194,-122.004672,2013,POINT (-2105837.76253553 4519725.204552194),NaN,NaN,NaN,NaN,NaN
4437,403475,4,17,S,ML,MD,2,6120.0,6097.900000,1141.555016,...,5.45,37.061194,-122.004672,2013,POINT (-2105837.76253553 4519725.204552194),NaN,NaN,NaN,NaN,NaN
4438,403475,4,17,S,ML,PM,2,7447.0,7321.744186,566.643421,...,5.45,37.061194,-122.004672,2013,POINT (-2105837.76253553 4519725.204552194),NaN,NaN,NaN,NaN,NaN


In [170]:
pems_use_ft_result_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 19131 entries, 0 to 19130
Data columns (total 28 columns):
station             19131 non-null int64
district            19131 non-null int64
route               19131 non-null int64
direction           19131 non-null object
type                19131 non-null object
time_period         19131 non-null object
lanes               19131 non-null int64
median_flow         19131 non-null float64
avg_flow            19131 non-null float64
sd_flow             19131 non-null float64
median_speed        5759 non-null float64
avg_speed           5759 non-null float64
sd_speed            5759 non-null float64
median_occup        19131 non-null float64
avg_occup           19131 non-null float64
sd_occupancy        19131 non-null float64
days_observed       19131 non-null int64
state_pm            19131 non-null float64
abs_pm              19131 non-null float64
latitude            19131 non-null float64
longitude           19131 non-null floa

In [168]:
pems_matched_ft_gdf

,station,longitude,latitude,access,area,bike_access,bridge,drive_access,est_width,fromIntersectionId,...,walk_access,wayId,width,id_y,geometry,tomtom_shieldnum,tomtom_rtedir,type,point,snap_distance
0,400009.0,-122.303083,37.863905,nan,nan,0,nan,1,nan,98fed6eab283a31a4ae23a5b28b94dbe,...,0,203395221,nan,f1d15c1424032c598afb43135b112b6f,"LINESTRING (-122.3049619 37.8702546, -122.3039...",580,E,ML,POINT (-122.303083 37.863905),0.000026
1,400060.0,-122.305289,37.870977,nan,nan,0,nan,1,nan,a8a09fb4394dfafe3e3135240e32c642,...,0,203395221,nan,073379dfd76e075d13a808774fb32a6e,"LINESTRING (-122.3056998 37.8725017, -122.3049...",580,E,ML,POINT (-122.305289 37.870977),0.000085
2,400108.0,-121.992771,37.190197,nan,nan,0,nan,1,nan,397e3a63e63d44e5f67526dcaaa3cce1,...,0,600135490,nan,e8032aefbf9b2a682aa5231e723917fd,"LINESTRING (-121.9921718 37.1728785, -121.9927...",17,,ML,POINT (-121.992771 37.190197),0.000039
3,400126.0,-122.301014,37.856366,nan,nan,0,nan,1,nan,f401c4af2679ccfb91b0908524ccf5ce,...,0,203317481,nan,6e834bfd77e7cada9627bff6552caebe,"LINESTRING (-122.3030235 37.86379609999999, -1...",580,E,ML,POINT (-122.301014 37.85636599999999),0.000067
4,400170.0,-122.230224,38.116679,nan,nan,0,nan,1,nan,b2f27f73790d742931e645dbef2d449d,...,0,123741362,nan,0d6e95237e3717ac19fd22a639b254e4,"LINESTRING (-122.2304763 38.1201127, -122.2304...",,,ML,POINT (-122.230224 38.116679),0.000009
5,400176.0,-122.303860,37.866548,nan,nan,0,nan,1,nan,98fed6eab283a31a4ae23a5b28b94dbe,...,0,203395221,nan,f1d15c1424032c598afb43135b112b6f,"LINESTRING (-122.3049619 37.8702546, -122.3039...",580,E,ML,POINT (-122.30386 37.866548),0.000039
6,400214.0,-122.545663,38.079686,nan,nan,0,nan,1,nan,d8bd3126d05b2ccbc18334969b49667f,...,0,394408678,nan,d58d64eac9c7823d516725cc1d100f59,"LINESTRING (-122.5450296 38.07905900000001, -1...",101,S,ML,POINT (-122.545663 38.079686),0.000016
7,400242.0,-122.291457,37.824311,"[nan, nan, nan]","[nan, nan, nan]",0,"[yes, yes, nan]",1,"[nan, nan, nan]",ee51d5d743bd4b73118244521fafada1,...,0,"[11197879, 11199418, 11194033]","[nan, nan, nan]",afba6ca2429c2f0f5cc0c3860c4a153d,"LINESTRING (-122.2912157 37.82523370000001, -1...",,,ML,POINT (-122.291457 37.824311),0.000104
8,400280.0,-121.858503,37.262054,"[nan, nan, nan]","[nan, nan, nan]",0,"[nan, yes, nan]",1,"[nan, nan, nan]",8750c60eaec4bf025ee2c1dfed17106d,...,0,"[184867239, 184867243, 184867246]","[nan, nan, nan]",145f9864a4a288cf667a4a3ad2ae276e,"LINESTRING (-121.8587791 37.2567994, -121.8587...",,,ML,POINT (-121.858503 37.262054),0.000023
9,400283.0,-122.530648,38.053731,"[nan, nan]","[nan, nan]",0,"[nan, nan]",1,"[nan, nan]",8ad496a93c5f733613f47bf7a7399053,...,0,"[394427797, 394427798]","[nan, nan]",d25e405f30b42f82b424d14ea094068c,"LINESTRING (-122.5311171 38.0510051, -122.5310...",101,S,ML,POINT (-122.530648 38.053731),0.000027


In [147]:
unique_pems_nearest_not_matched_df

,station,district,route,direction,type,time_period,lanes,median_flow,avg_flow,sd_flow,...,abs_pm,latitude,longitude,year,geometry,shstReferenceId,roadway,tomtom_shieldnum,tomtom_rtedir,snap_distance
15,400009,4,80,W,ML,AM,5,16085.0,15930.111111,577.366524,...,10.940,37.863905,-122.303083,2005,POINT (-2101884.901027649 4620392.963378702),NaN,NaN,NaN,NaN,NaN
106,400060,4,80,W,ML,AM,5,11553.0,11468.783784,404.328946,...,11.450,37.870977,-122.305289,2005,POINT (-2101806.466966349 4621266.028712609),NaN,NaN,NaN,NaN,NaN
196,400108,4,17,N,ML,AM,2,11681.0,11593.333333,415.050969,...,16.613,37.190197,-121.992771,2005,POINT (-2099822.711418324 4534079.906411736),NaN,NaN,NaN,NaN,NaN
243,400126,4,80,W,ML,AM,5,16225.0,15834.580645,1858.018403,...,10.400,37.856366,-122.301014,2005,POINT (-2101993.916232532 4619471.014944267),NaN,NaN,NaN,NaN,NaN
317,400170,4,80,W,ML,AM,3,18775.5,18565.357143,730.574776,...,31.465,38.116679,-122.230224,2005,POINT (-2085409.637222063 4646866.960632916),NaN,NaN,NaN,NaN,NaN
332,400176,4,80,W,ML,AM,5,10729.0,10369.103448,1661.606287,...,11.130,37.866548,-122.303860,2005,POINT (-2101851.319719956 4620717.778582582),NaN,NaN,NaN,NaN,NaN
439,400214,4,101,N,ML,AM,3,15020.0,14167.937500,1809.445973,...,459.574,38.079686,-122.545663,2005,POINT (-2115199.882067111 4652514.170117799),NaN,NaN,NaN,NaN,NaN
486,400242,4,880,S,ML,AM,4,13825.0,13767.444444,328.465424,...,45.127,37.824311,-122.291457,2005,POINT (-2102387.849476384 4615527.157334428),NaN,NaN,NaN,NaN,NaN
576,400280,4,87,N,ML,AM,2,7059.0,7120.066667,391.753540,...,0.450,37.262054,-121.858503,2005,POINT (-2084837.757232615 4538177.787857929),NaN,NaN,NaN,NaN,NaN
581,400283,4,101,N,ML,AM,4,12995.0,12995.953488,319.527401,...,457.564,38.053731,-122.530648,2005,POINT (-2114882.088257055 4649091.826510089),NaN,NaN,NaN,NaN,NaN


In [144]:
links_within_buffer_gdf

,access,area,bike_access,bridge,drive_access,est_width,fromIntersectionId,highway,id_x,junction,...,id_y,geometry,tomtom_shieldnum,tomtom_rtedir,station,type,latitude,longitude,point,snap_distance
226554,nan,nan,0,nan,1,nan,98fed6eab283a31a4ae23a5b28b94dbe,motorway,f1d15c1424032c598afb43135b112b6f,nan,...,f1d15c1424032c598afb43135b112b6f,"LINESTRING (-122.3049619 37.8702546, -122.3039...",580,E,400009,ML,37.863905,-122.303083,POINT (-122.303083 37.863905),0.000026
467450,nan,nan,0,nan,1,nan,f401c4af2679ccfb91b0908524ccf5ce,motorway,6e834bfd77e7cada9627bff6552caebe,nan,...,6e834bfd77e7cada9627bff6552caebe,"LINESTRING (-122.3030235 37.86379609999999, -1...",580,E,400009,ML,37.863905,-122.303083,POINT (-122.303083 37.863905),0.000124
421436,"[nan, nan, nan]","[nan, nan, nan]",1,"[nan, nan, nan]",1,"[nan, nan, nan]",2b1972a5f2a5271a6b03135da4b1ce4f,"[tertiary, tertiary, tertiary]",2d5afaf4a0da0a8f25395f1b4568d9c8,"[nan, nan, nan]",...,2d5afaf4a0da0a8f25395f1b4568d9c8,"LINESTRING (-122.3051319 37.86567460000001, -1...",,,400009,ML,37.863905,-122.303083,POINT (-122.303083 37.863905),0.000238
1001345,"[nan, nan, nan]","[nan, nan, nan]",1,"[nan, nan, nan]",1,"[nan, nan, nan]",ef6d52098968ad9acfc4b83a52b14863,"[tertiary, tertiary, tertiary]",2d5afaf4a0da0a8f25395f1b4568d9c8,"[nan, nan, nan]",...,2d5afaf4a0da0a8f25395f1b4568d9c8,"LINESTRING (-122.3051319 37.86567460000001, -1...",,,400009,ML,37.863905,-122.303083,POINT (-122.303083 37.863905),0.000238
741993,nan,nan,0,nan,1,nan,e1f41f0556bda3b2bb00e498514a68e1,motorway,9587d1507685606aa72cb863137d4497,nan,...,9587d1507685606aa72cb863137d4497,"LINESTRING (-122.3027738 37.86382889999999, -1...",580,W,400009,ML,37.863905,-122.303083,POINT (-122.303083 37.863905),0.000279
799056,nan,nan,0,nan,1,nan,02919934e6110e393e56c0d39ed555f8,motorway,15db425b10d8f8ba5de83f3ff39e6b6c,nan,...,15db425b10d8f8ba5de83f3ff39e6b6c,"LINESTRING (-122.2995568 37.8521718, -122.3003...",580,W,400009,ML,37.863905,-122.303083,POINT (-122.303083 37.863905),0.000318


In [119]:
stop_buffer_df = row_df.copy()
stop_buffer_df["geometry"] = stop_buffer_df.apply(lambda x: geodesic_point_buffer(x.latitude, x.longitude, 5), 
                                                      axis = 1)

stop_buffer_df = gpd.GeoDataFrame(stop_buffer_df, geometry = stop_buffer_df["geometry"], crs = {'init' : 'epsg:4326'})

stop_buffer_link_df = gpd.sjoin(link, stop_buffer_df[["geometry", "type"]], 
                                             how = "left", op = "intersects")

In [122]:
stop_buffer_link_df[stop_buffer_link_df["type"].notnull()].shstReferenceId

226554    3fe69a55506d7a078c327d075af084c8
Name: shstReferenceId, dtype: object

In [111]:
stop_buffer_df

,type,latitude,longitude,geometry
15,ML,37.863905,-122.303083,POLYGON ((-122.3030261782874 37.86390499998629...


In [88]:
unique_pems_nearest_not_matched_df.iloc[[0]]

,station,district,route,direction,type,time_period,lanes,median_flow,avg_flow,sd_flow,...,abs_pm,latitude,longitude,year,geometry,shstReferenceId,roadway,tomtom_shieldnum,tomtom_rtedir,snap_distance
15,400009,4,80,W,ML,AM,5,16085.0,15930.111111,577.366524,...,10.94,37.863905,-122.303083,2005,POINT (-2101884.901027649 4620392.963378702),NaN,NaN,NaN,NaN,NaN


In [54]:
pems_nearest_not_matched_df = pems_nearest_not_matched_df.round({"longitude" : 6, "latitude" : 6})

In [55]:
pems_use_shst_result_df = pd.merge(pems_nearest_not_matched_df.drop("shstReferenceId", axis = 1),
                                   pems_shst_match_df[["shstReferenceId", 'station', 'longitude', 'latitude', 'source']],
                                   how = "left",
                                   on = ['station', 'longitude', 'latitude'])

In [56]:
pems_use_shst_result_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 7828 entries, 0 to 7827
Data columns (total 28 columns):
station             7828 non-null int64
district            7828 non-null int64
route               7828 non-null int64
direction           7828 non-null object
type                7828 non-null object
time_period         7828 non-null object
lanes               7828 non-null int64
median_flow         7828 non-null float64
avg_flow            7828 non-null float64
sd_flow             7828 non-null float64
median_speed        6668 non-null float64
avg_speed           6668 non-null float64
sd_speed            6668 non-null float64
median_occup        7828 non-null float64
avg_occup           7828 non-null float64
sd_occupancy        7828 non-null float64
days_observed       7828 non-null int64
state_pm            7828 non-null float64
abs_pm              7828 non-null float64
latitude            7828 non-null float64
longitude           7828 non-null float64
year            

In [57]:
pems_use_shst_result_df.station.nunique()

393

# Concat match result from two methods

In [171]:
pems_nearest_matched_df = pems_nearest_gdf[pems_nearest_gdf.shstReferenceId.notnull()].copy()

In [59]:
pems_conflation_result_df = pd.concat(
        [pems_nearest_matched_df, pems_use_shst_result_df],
        sort = False,
        ignore_index = True
)

In [172]:
pems_conflation_result_df = pd.concat(
        [pems_nearest_matched_df, pems_use_ft_result_df],
        sort = False,
        ignore_index = True
)

In [173]:
pems_conflation_result_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 76057 entries, 0 to 76056
Data columns (total 28 columns):
station             76057 non-null int64
district            76057 non-null int64
route               76057 non-null int64
direction           76057 non-null object
type                76057 non-null object
time_period         76057 non-null object
lanes               76057 non-null int64
median_flow         76057 non-null float64
avg_flow            76057 non-null float64
sd_flow             76057 non-null float64
median_speed        60350 non-null float64
avg_speed           60350 non-null float64
sd_speed            60350 non-null float64
median_occup        76057 non-null float64
avg_occup           76057 non-null float64
sd_occupancy        76057 non-null float64
days_observed       76057 non-null int64
state_pm            76057 non-null float64
abs_pm              76057 non-null float64
latitude            76057 non-null float64
longitude           76057 non-null f

In [315]:
pems_conflation_result_df["source"].fillna("nearest", inplace = True)

In [176]:
pems_conflation_result_df= pems_conflation_result_df.to_crs(epsg = 4326)

In [178]:
pems_conflation_result_df.to_file(data_interim_dir + "mtc/pems_conflation_result_new.geojson",
                                  driver = "GeoJSON")

In [179]:
pems_conflation_result_df.head(3)

,station,district,route,direction,type,time_period,lanes,median_flow,avg_flow,sd_flow,...,abs_pm,latitude,longitude,year,geometry,shstReferenceId,roadway,tomtom_shieldnum,tomtom_rtedir,snap_distance
0,400000,4,101,S,ML,AM,3,16383.0,16237.843137,691.559263,...,459.52,38.081167,-122.547606,2005,POINT (-122.547606 38.081167),df209cac09d05a2f7e0a22dc8e931df8,motorway,101,S,3.17488
1,400000,4,101,S,ML,EA,3,4366.5,4376.357143,102.251993,...,459.52,38.081167,-122.547606,2005,POINT (-122.547606 38.081167),df209cac09d05a2f7e0a22dc8e931df8,motorway,101,S,3.17488
2,400000,4,101,S,ML,EV,3,8523.0,8434.255814,420.135754,...,459.52,38.081167,-122.547606,2005,POINT (-122.547606 38.081167),df209cac09d05a2f7e0a22dc8e931df8,motorway,101,S,3.17488


In [61]:
pems_df.station.nunique()

3802

In [62]:
3332 + 393

3725

In [64]:
pems_gdf.station.nunique()

3700

In [65]:
pems_conflation_result_df.station.nunique()

3700

In [69]:
pems_conflation_result_df.groupby(["shstReferenceId", "station", "year"]).count()

district  route  direction  \
shstReferenceId                  station year                               
000619909cfedfdc3ae846759247e09f 404433  2014         5      5          5   
                                 407341  2016         5      5          5   
                                         2017         5      5          5   
001faa1f8bf0fafb4298b7438a83b506 400615  2005         5      5          5   
                                         2006         5      5          5   
                                         2007         5      5          5   
                                         2008         5      5          5   
                                         2009         5      5          5   
                                         2010         5      5          5   
                                         2011         5      5          5   
                                         2013         5      5          5   
                                         2014         5      5          5   
                                         2015         5      5          5   
                                         2016         5      5          5   
                                         2017        10     10         10   
                                 414478  2017         5      5          5   
                                 414479  2017         5      5          5   
                                 414480  2017         5      5          5   
                                 419158  2017         1      1          1   
                                 419159  2017         2      2          2   
00441ada4f77ea0e20dd17acd688e289 404649  2014         5      5          5   
                                         2015         5      5          5   
005586c0c986e59d4dcd979513f7e8e6 401943  2014         5      5          5   
                                         2015         5      5          5   
                                         2016         5      5          5   
                                         2017         5      5          5   
                                 402067  2014         5      5          5   
                                         2015         5      5          5   
                                         2016         5      5          5   
                                         2017         5      5          5   
...                                                 ...    ...        ...   
ff90d3fb5034f779cc6c4841f2fbcfc5 402096  2017         5      5          5   
                                 408152  2015         5      5          5   
                                         2016         5      5          5   
                                         2017         5      5          5   
                                 408153  2015         5      5          5   
                                         2016         5      5          5   
                                         2017         5      5          5   
                                 408157  2016         5      5          5   
                                         2017         5      5          5   
ff9ba0a399cb6a07872339ed82edea56 400703  2016         5      5          5   
                                         2017         5      5          5   
                                 403325  2013         5      5          5   
                                         2014         5      5          5   
                                         2015         5      5          5   
                                         2016         5      5          5   
                                         2017         5      5          5   
                                 412871  2016         5      5          5   
                                         2017         5      5          5   
ff9fb0954535136f65fa9fda0774b468 404877  2013         5      5          5   
                                         20